In [1]:
import os
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
import multiprocessing as mp
from functools import partial
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch.optim import Adam
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

In [2]:
trained_model_path = '/workdir/security/home/junjiehuang2468/paper/trained_models_weight/ember/'
best_trained_model = '2022-01-18 14:55/2w_epoch:0_test_acc:0.890858.pt'
data_path = "/workdir/security/home/junjiehuang2468/paper/data/ember2018/"
train_data_path = data_path + "malwares/" 
test_data_path = data_path + "test_malwares/" 

In [3]:
CUDA = True if torch.cuda.is_available() else False
NUM_WORKERS = 24
BATCH_SIZE = 20
LEAVE_BIT_NUMBER = 20000
KERNEL_SIZE = 500

In [4]:
LEAVE_BIT_NUMBER_DICT = {10000:'1w',20000:'2w'}

In [5]:
trainset = pd.read_csv(data_path + 'train_dataset.csv')
validset = pd.read_csv(data_path + 'valid_dataset.csv')
testset = pd.read_csv(data_path + 'test_dataset.csv')
testset = testset.iloc[np.argwhere(testset['labels'].values == 1).squeeze(),:]

In [6]:
class ExeDataset(Dataset): # ExeDataset不是我取的，要記得改
    def __init__(self, malware_names, data_path, labels, leave_bit_num):
        self.malware_names = malware_names
        self.data_path = data_path
        self.labels = labels
        self.leave_bit_num = leave_bit_num

    def __len__(self):
        return len(self.malware_names)

    def __getitem__(self, idx):
        with open(self.data_path + self.malware_names[idx] + '.txt','rb') as fp:
            data = [bit+1 for bit in fp.read()[:self.leave_bit_num]]
            padding = [0]*(self.leave_bit_num-len(data))
            data = data + padding

        return np.array(data), np.array([self.labels[idx]])

In [7]:
train_dataset = ExeDataset(
    trainset["id"].tolist(), 
    train_data_path, 
    trainset["labels"].tolist(), 
    LEAVE_BIT_NUMBER
)
valid_dataset = ExeDataset(
    validset["id"].tolist(), 
    train_data_path, 
    validset["labels"].tolist(), 
    LEAVE_BIT_NUMBER
)
test_dataset = ExeDataset(
    testset["id"].tolist(), 
    test_data_path, 
    testset["labels"].tolist(), 
    LEAVE_BIT_NUMBER
)

In [8]:
trainloader = DataLoader(
    dataset = train_dataset,
    batch_size = BATCH_SIZE,
    shuffle = False,
    num_workers = NUM_WORKERS,
    pin_memory = True
)
validloader = DataLoader(
    dataset = valid_dataset,
    batch_size = BATCH_SIZE,
    shuffle = False,
    num_workers = NUM_WORKERS,
    pin_memory = True
)
testloader = DataLoader(
    dataset = test_dataset,
    batch_size = BATCH_SIZE,
    shuffle = False,
    num_workers = NUM_WORKERS,
    pin_memory = True
)

In [9]:
class Model(nn.Module):
    def __init__(self, data_length = 2e6, kernel_size = 500):
        super().__init__()
        self.embedding = nn.Embedding(257, 8, padding_idx=0)
        self.conv_layer_1 = nn.Conv1d(4, 128, kernel_size, stride = kernel_size, bias = True)
        # self.bn_1 = nn.BatchNorm1d(128)
        self.conv_layer_2 = nn.Conv1d(4, 128, kernel_size, stride = kernel_size, bias = True)
        self.pool_layer_2 = nn.MaxPool1d(data_length//kernel_size)
        self.fc_layer_3 = nn.Linear(128, 128)
        self.fc_layer_4 = nn.Linear(128, 2)
        
    def forward(self,x):
        embedd_x = self.embedding(x)
        # embedd_x.requires_grad = True
        embedd_x.retain_grad()
        x = embedd_x.transpose(-1,-2)
        x_conv_1 = self.conv_layer_1(x[:,:4,:])
        x_conv_2 = torch.sigmoid(self.conv_layer_2(x[:,4:,:]))
        x = x_conv_1*x_conv_2
        del x_conv_1,x_conv_2
        x = self.pool_layer_2(x).squeeze()
        x = self.fc_layer_3(x)
        x = self.fc_layer_4(x)
        return x,embedd_x

In [10]:
def mp_func(idx,one_of_all_embedd,attacked_embedd_x):
    temp = np.linalg.norm(attacked_embedd_x - one_of_all_embedd,ord=2,axis=-1)
    temp = np.expand_dims(temp, axis=-1)
    return [idx,temp]

In [11]:
model = Model(data_length=LEAVE_BIT_NUMBER,kernel_size=KERNEL_SIZE)

ce_loss = nn.CrossEntropyLoss()
optim = Adam(model.parameters())

model = model.cuda() if CUDA else model
ce_loss = ce_loss.cuda() if CUDA else ce_less

In [12]:
model.load_state_dict(torch.load(trained_model_path + best_trained_model))

<All keys matched successfully>

In [13]:
from collections import defaultdict
dic = defaultdict(list)

In [ ]:
model.eval()
total_acc = []
# bar = tqdm(validloader)
bar = tqdm(testloader)
for step, (batch_data,batch_label) in enumerate(bar):
    batch_data = batch_data.cuda() if CUDA else batch_data
    clone_batch_data = batch_data.detach().clone()
    batch_label = batch_label.cuda() if CUDA else batch_label
    batch_label = batch_label.squeeze()

    optim.zero_grad()
    temp_label = torch.zeros((len(batch_label),2)).cuda()
    temp_label[:,0] = 1
    temp_total_acc = []
    pred,embedd_x = model(batch_data)
    _, predicted = torch.max(pred, 1)
    acc = (batch_label.cpu().data.numpy() == predicted.cpu().data.numpy()).mean()
    temp_total_acc.append(acc)

    mask = (batch_data == 0).float()
    random_padding = torch.randint_like(input=mask.float(),low=1,high=257) * mask
    batch_data += random_padding.long()

    pred,embedd_x = model(batch_data)
    _, predicted = torch.max(pred, 1)
    acc = (batch_label.cpu().data.numpy() == predicted.cpu().data.numpy()).mean()
    temp_total_acc.append(acc)
    for _ in range(8):
        pred,embedd_x = model(batch_data)
        loss = ce_loss(pred, temp_label)
        loss.backward()

        attacked_embedd_x = embedd_x - 0.5*embedd_x.grad.sign()
        all_embedd = model.embedding(torch.arange(start=0,end=257).cuda())

        shape = attacked_embedd_x.shape
        attacked_embedd_x = attacked_embedd_x.reshape((shape[0],shape[1],1,shape[2]))
        attacked_embedd_x = attacked_embedd_x.expand(shape[0],shape[1],257,shape[2])

        results = torch.linalg.norm(attacked_embedd_x - all_embedd,ord=2,dim=-1)
        batch_data = ((1-mask)*batch_data + mask*torch.argmin(results,dim=-1)).long()

        pred,embedd_x = model(batch_data)
        _, predicted = torch.max(pred, 1)
        acc = (batch_label.cpu().data.numpy() == predicted.cpu().data.numpy()).mean()
        temp_total_acc.append(acc)

    pred = pred.detach().cpu().numpy()
    batch_label = batch_label.detach().cpu().numpy()
    pred = np.argmax(pred,1)
    temp_acc = (batch_label == pred).mean()
    temp = batch_label == pred
    if temp.mean() != 0:
        idxs = np.argwhere(temp).reshape(-1)
        for idx in idxs:
            print(step*BATCH_SIZE+idx)
            padding_num = (clone_batch_data[idx] == 0).sum().tolist()
            print(padding_num)
            dic[padding_num].append(step*BATCH_SIZE+idx)

    total_acc.append(temp_total_acc)
    total_acc_str = '[' + ' '.join(map(lambda x: '%.9f'%x,np.mean(total_acc,axis=0))) + ']'
    bar.set_description(total_acc_str)


[0.873076923 0.430769231 0.003846154 0.003846154 0.003846154 0.003846154 0.003846154 0.003846154 0.003846154 0.003846154]:   0%|          | 13/5000 [00:05<21:20,  3.90it/s] 

253
0


[0.876190476 0.433333333 0.004761905 0.004761905 0.004761905 0.004761905 0.004761905 0.004761905 0.004761905 0.004761905]:   0%|          | 21/5000 [00:07<20:41,  4.01it/s]

400
0


[0.862121212 0.443939394 0.004545455 0.004545455 0.004545455 0.004545455 0.004545455 0.004545455 0.004545455 0.004545455]:   1%|          | 33/5000 [00:10<20:44,  3.99it/s]

...

[0.858725762 0.436426593 0.005318560 0.005290859 0.005290859 0.005290859 0.005290859 0.005290859 0.005290859 0.005290859]:  36%|███▌      | 1805/5000 [07:50<14:04,  3.78it/s]

36096
0


[0.858595909 0.436484245 0.005334439 0.005306799 0.005306799 0.005306799 0.005306799 0.005306799 0.005306799 0.005306799]:  36%|███▌      | 1809/5000 [07:51<14:06,  3.77it/s]

36163
0


[0.858727974 0.436563877 0.005313877 0.005286344 0.005286344 0.005286344 0.005286344 0.005286344 0.005286344 0.005286344]:  36%|███▋      | 1816/5000 [07:53<14:02,  3.78it/s]

In [ ]:
# import pickle

In [20]:
# with open('/workdir/security/home/junjiehuang2468/paper/results/ember/XAI/my_way_dic.txt','rb') as fp:
#     my_way_dic = pickle.load(fp)

In [17]:
# with open('/workdir/security/home/junjiehuang2468/paper/results/ember/XAI/PGD_dic.txt','wb') as fp:
#     pickle.dump(dic,fp)